Mount Google Drive (optional)


In [365]:
# from google.colab import drive
# drive.mount('/content/drive')

# import os
# # os.chdir("/content/drive/MyDrive/....")  # file path
# print(os.getcwd())


# **HW2 : Decision Tree and Random Forest**

In _assignment 2_, you need to finish :

1. Basic Part : Implement a **Decision Tree** model and predict whether the patients in the validation set have diabetes

   > - Step 1 : Load the input data
   > - Step 2 : Calculate the Entropy and Information Gain
   > - Step 3 : Find the Best Split
   > - Step 4 : Split into 2 branches
   > - Step 5 : Build decision tree
   > - Step 6 : Save the answers from step2 to step5
   > - Step 7 : Split data into training set and validation set
   > - Step 8 : Train a decision tree model with training set
   > - Step 9 : Predict the cases in the _validation set_ by using the model trained in _Step8_
   > - Step 10 : Calculate the f1-score of your predictions in _Step9_
   > - Step 11 : Write the Output File

2. Advanced Part : Build a **Random Forest** model to make predictions
   > - Step 1 : Load the input data
   > - Step 2 : Load the test data
   > - Step 3 : Build a random forest
   > - Step 4 : Predict the cases in the test data by using the model trained in _Step3_
   > - Step 5 : Save the predictions(from _Step 4_) in a csv file


# **Basic Part** (60%)

In this part, your need to implement a Decision Tree model by completing the following given functions.

Also, you need to run these functions with the given input variables and save the output in a csv file **hw2_basic.csv**


## Import Packages

> Note : You **cannot** import any other packages in both basic part and advanced part


In [366]:
import numpy as np
import pandas as pd
import math
import random
from numpy import sqrt
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score


## Step1: Load the input data

First, load the input file **hw2_input_basic.csv**


In [367]:
input_data = pd.read_csv('hw2_input_basic.csv')
input_data


,age,bmi,gender,height,weight,glucose_apache,heart_rate_apache,resprate_apache,sodium_apache,diabetes_mellitus
0,70.0,25.984659,1,172.7,77.50,116.0,101.0,49.0,137.0,0
1,30.0,31.310368,1,170.2,90.70,71.0,39.0,33.0,144.0,0
2,54.0,24.388824,1,177.8,77.10,120.0,120.0,31.0,141.0,0
3,65.0,34.141074,0,170.2,98.90,73.0,48.0,36.0,140.0,1
4,49.0,22.564743,1,172.7,67.30,207.0,119.0,6.0,144.0,0
5,62.0,29.424010,0,154.9,70.60,113.0,60.0,32.0,137.0,0
6,85.0,27.673574,1,154.9,66.40,102.0,49.0,36.0,142.0,0
7,65.0,22.269432,1,177.8,70.40,333.0,59.0,6.0,145.0,1
8,85.0,35.879362,0,165.1,97.80,124.0,92.0,30.0,136.0,0
9,81.0,20.859375,0,160.0,53.40,136.0,118.0,52.0,138.0,0


## Global attributes

Define the global attributes

> Note : You **cannot** modify the values of these attributes we given in the basic part


In [368]:
max_depth = 2
depth = 0
min_samples_split = 2
n_features = input_data.shape[1] - 1


> You can add your own global attributes here


In [369]:
Y_FEATURE = 'diabetes_mellitus'


## Step2 : Calculate the Entropy and Information Gain

Calculate the information gain and entropy values before separate data into left subtree and right subtree


In [370]:
def entropy(data):
  """
  This function measures the amount of uncertainty in a probability distribution
  args: 
  * data(type: DataFrame): the data you're calculating for the entropy
  return:
  * entropy_value(type: float): the data's entropy
  """

  p = data[Y_FEATURE].sum(axis=0) / data[Y_FEATURE].shape[0]
  entropy_value = 0
  if (p != 1 and p != 0):
    entropy_value = -p * math.log2(p) - (1 - p) * math.log2(1 - p)

  return entropy_value


# [Note] You have to save the value of "ans_entropy" into the output file
ans_entropy = entropy(input_data)
print("ans_entropy = ", ans_entropy)


ans_entropy =  0.9871377743721863


In [371]:
def information_gain(data, mask):
  """
  This function will calculate the information gain
  args:
  * data(type: DataFrame): the data you're calculating for the information gain
  * mask(type: Series): partition information(left/right) of current input data, 
    - boolean 1(True) represents split to left subtree
    - boolean 0(False) represents split to right subtree
  return:
  * ig(type: float): the information gain you can obtain by classify data with this given mask
  """

  entropy_before = entropy(data)
  left_ratio = mask.sum(axis=0) / mask.shape[0]
  entropy_after = left_ratio * entropy(data[mask]) + (1 - left_ratio) * entropy(data[~mask])
  ig = entropy_before - entropy_after

  return ig


# [Note] You have to save the value of "ans_informationGain" into your output file
temp1 = np.zeros((int(input_data.shape[0] / 4), 1), dtype=bool)
temp2 = np.ones(((input_data.shape[0] - int(input_data.shape[0] / 4), 1)), dtype=bool)
temp_mask = np.concatenate((temp1, temp2))
df_mask = pd.DataFrame(temp_mask, columns=['mask'])
ans_informationGain = information_gain(input_data, df_mask['mask'])
print("ans_informationGain = ", ans_informationGain)


ans_informationGain =  0.0834598868480716


## Step3 : Find the Best Split

Find the best split combination, **feature** and **threshold**, by calculating the information gain


In [372]:
def find_best_split(data):
  """
  This function will find the best split combination of data
  args:
  * data(type: DataFrame): the input data
  return
  * best_ig(type: float): the best information gain you obtain
  * best_threshold(type: float): the value that splits data into 2 branches
  * best_feature(type: string): the feature that splits data into 2 branches
  """

  best_ig = 0
  best_threshold = 0
  best_feature = 0

  # All
  for feature in list(data.columns):
    if (feature == Y_FEATURE):
      continue
    thresholds = data[feature].unique()
    thresholds = np.sort(thresholds)
    thresholds = (thresholds[:-1] + thresholds[1:]) / 2
    for threshold in thresholds:
      mask_leq = data[feature] <= threshold
      if (mask_leq.sum() == 0 or (~mask_leq).sum() == 0):
        continue
      ig = information_gain(data, mask_leq)
      if (ig > best_ig):
        best_ig = ig
        best_threshold = threshold
        best_feature = feature
  
  # for feature in list(data.columns):
  #   if (feature == Y_FEATURE):
  #     continue
  #   thresholds = data[feature].quantile(np.arange(start=step, stop=1, step=step))
  #   for threshold in thresholds:
  #     mask_leq = data[feature] <= threshold
  #     if (mask_leq.sum() == 0 or (~mask_leq).sum() == 0):
  #       continue
  #     ig = information_gain(data, mask_leq)
  #     if (ig > best_ig):
  #       best_ig = ig
  #       best_threshold = threshold
  #       best_feature = feature

  return best_ig, best_threshold, best_feature

# [Note] You have to save the value of "ans_ig", "ans_value", and "ans_name" into the output file
ans_ig, ans_value, ans_name = find_best_split(input_data)
print("ans_ig = ", ans_ig)
print("ans_value = ", ans_value)
print("ans_name = ", ans_name)


ans_ig =  0.3522950515812332
ans_value =  235.5
ans_name =  glucose_apache


## Step4 : Split into 2 branches

Using the best split combination you find in function _find_best_split()_ to split data into Left Subtree and Right Subtree


In [373]:
def make_partition(data, feature, threshold):
  """
  This function will split the data into 2 branches
  args:
  * data(type: DataFrame): the input data
  * feature(type: string): the attribute(column name)
  * threshold(type: float): the threshold for splitting the data
  return:
  * left(type: DataFrame): the divided data that matches(less than or equal to) the assigned feature's threshold
  * right(type: DataFrame): the divided data that doesn't match the assigned feature's threshold
  """

  mask_leq = data[feature] <= threshold
  left = data[mask_leq]
  right = data[~mask_leq]

  return left, right


# [Note] You have to save the value of "ans_left" into the output file
left, right = make_partition(input_data, 'age', 61.0)
ans_left = left.shape[0]
print("ans_left = ", ans_left)


ans_left =  10


## Step5 : Build Decision Tree

Use the above functions to implement the decision tree

Instructions:

1.  If current depth < max_depth and the remaining number of samples > min_samples_split: continue to classify those samples
2.  Use function _find_best_split()_ to find the best split combination
3.  If the obtained information gain is **greater than 0**: can build a deeper decision tree (add depth)
4.  Use function _make_partition()_ to split the data into two parts
5.  Save the features and corresponding thresholds (starting from the root) used by the decision tree into _ans_features[]_ and _ans_thresholds[]_ respectively


In [374]:
def build_tree(data, max_depth, min_samples_split, depth):
  """
  This function will build the decision tree
  args:
  * data(type: DataFrame): the data you want to apply to the decision tree
  * max_depth: the maximum depth of a decision tree
  * min_samples_split: the minimum number of instances required to do partition
  * depth: the height of the current decision tree
  return:
  * subtree: the decision tree structure including root, branch, and leaf (with the attributes and thresholds)
  """

  # check the condition of current depth and the remaining number of samples
  if depth < max_depth and data.shape[0] >= min_samples_split:
    # call find_best_split() to find the best combination
    ig, threshold, feature = find_best_split(data)
    # check the value of information gain is greater than 0 or not
    if ig > 0:
      # update the depth
      # call make_partition() to split the data into two parts
      left_data, right_data = make_partition(data, feature, threshold)
      # If there is no data split to the left tree OR no data split to the left tree
      if left_data.shape[0] == 0 or right_data.shape[0] == 0:
        # return the label of the majority
        label = int(data[Y_FEATURE].sum() > (~data[Y_FEATURE].astype(bool)).sum())
        return label
      else:
        question = "{} {} {}".format(feature, "<=", threshold)
        subtree = {question: []}

        # call function build_tree() to recursively build the left subtree and right subtree
        left_subtree = build_tree(left_data, max_depth, min_samples_split, depth + 1)
        right_subtree = build_tree(right_data, max_depth, min_samples_split, depth + 1)
        if left_subtree == right_subtree:
          subtree = left_subtree
        else:
          subtree[question].append(left_subtree)
          subtree[question].append(right_subtree)
    else:
      # return the label of the majority
      label = int(data[Y_FEATURE].sum() > (~data[Y_FEATURE].astype(bool)).sum())
      return label
  else:
    # return the label of the majority
    label = int(data[Y_FEATURE].sum() > (~data[Y_FEATURE].astype(bool)).sum())
    return label

  return subtree


An example of the output from _build_tree()_

```
{'bmi <= 33.5': [1, {'age <= 68.5': [0, 1]}]}
```

Therefore,

```
ans_features = ['bmi', 'age']
ans_thresholds = [33.5, 68.5]
```


In [375]:
ans_features = []
ans_thresholds = []

decisionTree = build_tree(input_data, max_depth, min_samples_split, depth)
decisionTree


{'glucose_apache <= 235.5': [{'heart_rate_apache <= 143.5': [0, 1]}, 1]}

In [376]:
# [Note] You have to save the features in the "decisionTree" structure (from root to branch and leaf) into the output file
ans_features = []

cur_decisionTree = decisionTree
while (isinstance(cur_decisionTree, dict)):
  ans_features.append(list(cur_decisionTree.keys())[0].split()[0])
  left_tree = list(cur_decisionTree.values())[0][0]
  right_tree = list(cur_decisionTree.values())[0][1]
  if isinstance(right_tree, dict):
    cur_decisionTree = right_tree
  elif isinstance(left_tree, dict):
    cur_decisionTree = left_tree
  else:
    break

print(ans_features)


['glucose_apache', 'heart_rate_apache']


In [377]:
# [Note] You have to save the corresponding thresholds for the features in the "ans_features" list into the output file
ans_thresholds = []

cur_decisionTree = decisionTree
while (isinstance(cur_decisionTree, dict)):
  ans_thresholds.append(list(cur_decisionTree.keys())[0].split()[2])
  left_tree = list(cur_decisionTree.values())[0][0]
  right_tree = list(cur_decisionTree.values())[0][1]
  if isinstance(right_tree, dict):
    cur_decisionTree = right_tree
  elif isinstance(left_tree, dict):
    cur_decisionTree = left_tree
  else:
    break

print(ans_thresholds)


['235.5', '143.5']


## Step6 : Save answers


In [378]:
basic = []
basic.append(ans_entropy)
basic.append(ans_informationGain)
basic.append(ans_ig)
basic.append(ans_value)
basic.append(ans_name)
basic.append(ans_left)
for i in range(len(ans_features)):
  basic.append(ans_features[i])
for m in range(len(ans_thresholds)):
  basic.append(ans_thresholds[m])


## Step7 : Split data

Split data into training set and validation set

> Note: We have split the data into training set and validation. You **cannot** change the distribution of the data.


In [379]:
num_train = 20
num_validation = 10

training_data = input_data.iloc[:num_train]
validation_data = input_data.iloc[-num_validation:]

y_train = training_data[["diabetes_mellitus"]]
x_train = training_data.drop(['diabetes_mellitus'], axis=1)
y_validation = validation_data[["diabetes_mellitus"]]
x_validation = validation_data.drop(['diabetes_mellitus'], axis=1)
y_validation = y_validation.values.flatten()

print(input_data.shape)
print(training_data.shape)
print(validation_data.shape)


(30, 10)
(20, 10)
(10, 10)


## Step8 to Step10 : Make predictions with a decision tree


Define the attributions of the decision tree

> You **cannot** modify the values of these attributes in this part


In [380]:
max_depth = 2
depth = 0
min_samples_split = 2
n_features = x_train.shape[1]


We have finished the function '_classify_data()_' below, however, you can modify this function if you prefer completing it on your own way.


In [381]:
def classify_data(instance, tree):
  """
  This function will predict/classify the input instance
  args:
  * instance: a instance(case) to be predicted
  return:
  * answer: the prediction result (the classification result)
  """
  if (isinstance(tree, dict)):
    equation = list(tree.keys())[0]
    if equation.split()[1] == '<=':
      temp_feature = equation.split()[0]
      temp_threshold = equation.split()[2]
      if instance[temp_feature] > float(temp_threshold):
        answer = tree[equation][1]
      else:
        answer = tree[equation][0]
    else:
      if instance[equation.split()[0]] in (equation.split()[2]):
        answer = tree[equation][0]
      else:
        answer = tree[equation][1]

    if not isinstance(answer, dict):
      return answer
    else:
      return classify_data(instance, answer)
  elif (isinstance(tree, int)):
    return tree
  else:
    return random.randint(0, 1)


def make_prediction(tree, data):
  """
  This function will use your pre-trained decision tree to predict the labels of all instances in data
  args:
  * tree: the decision tree
  * data: the data to predict
  return:
  * y_prediction: the predictions
  """

  # [Note] You can call the function classify_data() to predict the label of each instance
  y_prediction = pd.DataFrame(
    np.zeros((data.shape[0], 1)), index=data.index, columns=[Y_FEATURE], dtype=int)

  for index in data.index:
    y_prediction.loc[index] = classify_data(data.loc[index], tree)

  return y_prediction.values.flatten()


def calculate_score(y_true, y_pred):
  """
  This function will calculate the f1-score of the predictions
  args:
  * y_true: the ground truth
  * y_pred: the predictions
  return:
  * score: the f1-score
  """

  score = f1_score(y_true, y_pred)

  return score


In [382]:
decision_tree = build_tree(training_data, max_depth, min_samples_split, depth)

y_pred = make_prediction(decision_tree, x_validation)

# [Note] You have to save the value of "ans_f1score" the your output file
ans_f1score = calculate_score(y_validation, y_pred)
print("ans_f1score = ", ans_f1score)


ans_f1score =  0.6666666666666666


## Step11 : Write the Output File

Save all of your answers in a csv file, named as **hw2_basic.csv**


In [383]:
ans_path = 'hw2_basic.csv'

# [Note] You have to save the value of "ans_f1score" into the output file
basic.append(ans_f1score)
print(basic)

pd.DataFrame(basic).to_csv(ans_path, header=None, index=None)


[0.9871377743721863, 0.0834598868480716, 0.3522950515812332, 235.5, 'glucose_apache', 10, 'glucose_apache', 'heart_rate_apache', '235.5', '143.5', 0.6666666666666666]


# **Advanced Part** (35%)


## Step1: Load the input data

First, load the input file **hw2_input_advanced.csv**


In [384]:
advanced_data = pd.read_csv('hw2_input_advanced.csv')


You can split _advanced_data_ into training set and validaiton set


In [385]:
advanced_data_size = advanced_data.shape[0]
validation_ratio = 0.2
num_train = math.ceil(advanced_data_size * (1 - validation_ratio))
num_validation = advanced_data_size - num_train
training_data = advanced_data.iloc[:num_train]
training_data = advanced_data
validation_data = advanced_data.iloc[-num_validation:]

y_train = training_data[[Y_FEATURE]]
x_train = training_data.drop([Y_FEATURE], axis=1)
y_validation = validation_data[[Y_FEATURE]]
x_validation = validation_data.drop([Y_FEATURE], axis=1)
y_validation = y_validation.values.flatten()


## Step2 : Load the test data

Load the input file **hw2_input_test.csv** to make predictions with the pre-trained random forest model


In [386]:
x_test = pd.read_csv('hw2_input_test.csv')
x_test

,age,bmi,gender,height,weight,arf_apache,bun_apache,creatinine_apache,gcs_eyes_apache,gcs_motor_apache,...,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,ventilated_apache,wbc_apache,apache_4a_hospital_death_prob,apache_4a_icu_death_prob
0,62,32.866392,1,177.80,103.9,1,31.0,10.30,4,6,...,36.4,0,157,26,134,36.1,0,4.56,0.06,0.03
1,82,23.582766,0,157.50,58.5,0,26.0,0.54,3,4,...,32.8,0,42,25,142,36.1,0,6.00,0.14,0.06
2,61,31.684520,1,172.70,94.5,0,16.0,1.11,4,6,...,35.3,0,129,6,131,36.8,0,8.59,0.05,0.03
3,58,45.156250,0,160.00,115.6,0,19.0,0.70,1,4,...,30.1,1,131,23,138,34.9,1,16.03,0.33,0.22
4,74,25.817016,1,172.70,77.0,0,25.0,0.93,4,6,...,34.5,0,55,12,135,36.3,0,45.80,0.12,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,73,17.943584,0,157.48,44.5,0,12.0,0.30,4,6,...,33.8,0,129,9,144,36.9,0,7.70,0.02,0.01
836,79,29.049732,1,167.60,81.6,0,48.0,2.19,4,6,...,42.7,0,163,9,139,36.4,0,10.77,0.06,0.03
837,85,24.627827,0,152.40,57.2,0,11.0,0.48,3,5,...,29.5,0,67,9,139,36.6,0,7.35,0.16,0.05
838,68,32.510940,1,193.00,121.1,0,14.0,0.64,4,6,...,37.5,0,61,10,140,36.9,1,16.02,0.00,0.00


## Step3 : Build a Random Forest


Define the attributions of the random forest

> - You **can** modify the values of these attributes in advanced part
> - Each tree can have different attribute values
> - There must be **at least** 3 decision trees in the random forest model
> - Must use function _build_tree()_ to build a random forest model
> - These are the parameters you can adjust :

    ```
    max_depth =
    depth = 0
    min_samples_split =

    # total number of trees in a random forest
    n_trees =

    # number of features to train a decision tree
    n_features =

    # the ratio to select the number of instances
    sample_size =
    n_samples = int(training_data.shape[0] * sample_size)
    ```


In [387]:
# Define the attributes

max_depth = 7
min_samples_split = 3

n_trees = 100
n_features = 16

sample_size = 0.6
n_samples = int(training_data.shape[0] * sample_size)

def attribute_randomize():
  global max_depth, n_features, n_samples
  max_depth = random.choice([3, 4, 4, 4, 5])
  n_features = random.choice([10, 12, 14, 15, 16, 18, 20])
  n_samples = random.choice([int(training_data.shape[0] * ss) for ss in [0.5, 0.7, 0.75, 1]])


In [388]:
import multiprocess as mp


def build_forest(data: pd.DataFrame, n_trees, n_features, n_samples):
  """
  This function will build a random forest.
  args:
  * data: all data that can be used to train a random forest
  * n_trees: total number of tree
  * n_features: number of features
  * n_samples: number of instances
  return:
  * forest: a random forest with 'n_trees' of decision tree
  """

  x_features = list(data.columns)
  x_features.remove(Y_FEATURE)
  indices = list(data.index)

  forest = []

  def build_partial_tree(idx):
    part_features = random.sample(x_features, n_features)  # Get random features from all x features
    part_features.append(Y_FEATURE)
    part_indices = random.sample(indices, n_samples)  # Get random indices
    tree = build_tree(data[part_features].loc[part_indices], max_depth, min_samples_split, 0)  # Reusing build_tree function
    return tree

  with mp.Pool(mp.cpu_count()) as p:
    forest = p.map(build_partial_tree, range(n_trees))

  return forest


In [389]:
forest = build_forest(training_data, n_trees, n_features, n_samples)


## Step4 : Make predictions with the random forest

> Note: Please print the f1-score of the predictions of each decision tree


In [390]:
def make_prediction_forest(forest, data: pd.DataFrame):
  """
  This function will use the pre-trained random forest to make the predictions
  args:
  * forest: the random forest
  * data: the data used to predict
  return:
  * y_prediction: the predicted results
  """

  y_prediction = pd.DataFrame(
    np.zeros((data.shape[0], 1)), index=data.index, columns=[Y_FEATURE], dtype=int)

  for tree in forest:
    for index in data.index:
      y_prediction.loc[index] += classify_data(data.loc[index], tree)

  y_prediction = (y_prediction / len(forest)) >= 0.5
  y_prediction = y_prediction.astype(int)

  return y_prediction.values.flatten()


In [391]:
for tree_idx, tree in zip(range(len(forest)), forest):
  y_pred = make_prediction(tree, x_validation)
  f1score = calculate_score(y_validation, y_pred)
  print(f'{tree_idx + 1}/{len(forest)} f1 score: {f1score}')

y_pred = make_prediction_forest(forest, x_validation)
f1score = calculate_score(y_validation, y_pred)
print(f'Total f1 score: {f1score}')


1/100 f1 score: 0.73046875
2/100 f1 score: 0.7342430149447693
3/100 f1 score: 0.72
4/100 f1 score: 0.7304582210242587
5/100 f1 score: 0.6440903054448872
6/100 f1 score: 0.6523994811932555
7/100 f1 score: 0.7414448669201521
8/100 f1 score: 0.7371836469824788
9/100 f1 score: 0.6629213483146066
10/100 f1 score: 0.7091152815013403
11/100 f1 score: 0.7463630613535738
12/100 f1 score: 0.6405740378343118
13/100 f1 score: 0.7348837209302326
14/100 f1 score: 0.7271531886916502
15/100 f1 score: 0.7490396927016645
16/100 f1 score: 0.7355163727959697
17/100 f1 score: 0.7238855622089154
18/100 f1 score: 0.7018716577540106
19/100 f1 score: 0.7059602649006623
20/100 f1 score: 0.6411804158283031
21/100 f1 score: 0.7414248021108178
22/100 f1 score: 0.6543367346938775
23/100 f1 score: 0.7077131258457375
24/100 f1 score: 0.7269102990033223
25/100 f1 score: 0.6996466431095406
26/100 f1 score: 0.7237715803452854
27/100 f1 score: 0.7525252525252525
28/100 f1 score: 0.6204136090727151
29/100 f1 score: 0.6735

In [392]:
max_f1 = 0
best_len = 0
tot_y_prediction = pd.DataFrame(np.zeros((x_validation.shape[0], 1)), index=x_validation.index, columns=[Y_FEATURE], dtype=int)
sum_y_prediction = pd.DataFrame(np.zeros((x_validation.shape[0], 1)), index=x_validation.index, columns=[Y_FEATURE], dtype=int)
for i, tree in zip(range(len(forest)), forest):
  y_prediction = pd.DataFrame(np.zeros((x_validation.shape[0], 1)), index=x_validation.index, columns=[Y_FEATURE], dtype=int)
  for index in x_validation.index:
    y_prediction.loc[index] += classify_data(x_validation.loc[index], tree)

  sum_y_prediction += y_prediction
  tot_y_prediction = (sum_y_prediction / (i + 1)) >= 0.5
  tot_y_prediction = tot_y_prediction.astype(int)

  f1score = calculate_score(y_validation, tot_y_prediction)
  if (f1score > max_f1):
    max_f1 = f1score
    best_len = i + 1

print(f'best f1 score: {max_f1}, {best_len}')


best f1 score: 0.7830065359477124, 74


In [ ]:
y_pred_test = make_prediction_forest(forest, x_test)


## Step5 : Write the Output File

Save your predictions from the **random forest** in a csv file, named as **hw2_advanced.csv**


In [ ]:
advanced = []
for i in range(len(y_pred_test)):
  advanced.append(y_pred_test[i])


In [ ]:
advanced_path = 'hw2_advanced.csv'
pd.DataFrame(advanced).to_csv(advanced_path, header=None, index=None)
